In [1]:
import lightningchart as lc
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from scipy.stats import gaussian_kde
from scipy.interpolate import griddata
import time

with open('D:/Computer Aplication/WorkPlacement/Projects/shared_variable.txt', 'r') as f:
    mylicensekey = f.read().strip()
lc.set_license(mylicensekey)

In [2]:
generation_data = pd.read_csv('D:/wenprograming23/src/team6/Implementation-of-a-Solar-Power-Modelling-Application-with-LightningChart-Python/Dataset/Plant_2_Generation_Data.csv')
weather_data = pd.read_csv('D:/wenprograming23/src/team6/Implementation-of-a-Solar-Power-Modelling-Application-with-LightningChart-Python/Dataset/Plant_2_Weather_Sensor_Data.csv')

generation_data['DATE_TIME'] = pd.to_datetime(generation_data['DATE_TIME'])
weather_data['DATE_TIME'] = pd.to_datetime(weather_data['DATE_TIME'])

print(generation_data.head())
print(weather_data.head())

   DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  DAILY_YIELD  \
0 2020-05-15   4136001  4UPUqMRk7TRMgml       0.0       0.0  9425.000000   
1 2020-05-15   4136001  81aHJ1q11NBPMrL       0.0       0.0     0.000000   
2 2020-05-15   4136001  9kRcWv60rDACzjR       0.0       0.0  3075.333333   
3 2020-05-15   4136001  Et9kgGMDl729KT4       0.0       0.0   269.933333   
4 2020-05-15   4136001  IQ2d7wF4YD8zU1Q       0.0       0.0  3177.000000   

    TOTAL_YIELD  
0  2.429011e+06  
1  1.215279e+09  
2  2.247720e+09  
3  1.704250e+06  
4  1.994153e+07  
            DATE_TIME  PLANT_ID       SOURCE_KEY  AMBIENT_TEMPERATURE  \
0 2020-05-15 00:00:00   4136001  iq8k7ZNt4Mwm3w0            27.004764   
1 2020-05-15 00:15:00   4136001  iq8k7ZNt4Mwm3w0            26.880811   
2 2020-05-15 00:30:00   4136001  iq8k7ZNt4Mwm3w0            26.682055   
3 2020-05-15 00:45:00   4136001  iq8k7ZNt4Mwm3w0            26.500589   
4 2020-05-15 01:00:00   4136001  iq8k7ZNt4Mwm3w0            26.596148 

In [3]:
merged_data = pd.merge(generation_data, weather_data, on=['DATE_TIME', 'PLANT_ID'])
merged_data['DATE_TIME'] = pd.to_datetime(merged_data['DATE_TIME'])
print(merged_data.head())


   DATE_TIME  PLANT_ID     SOURCE_KEY_x  DC_POWER  AC_POWER  DAILY_YIELD  \
0 2020-05-15   4136001  4UPUqMRk7TRMgml       0.0       0.0  9425.000000   
1 2020-05-15   4136001  81aHJ1q11NBPMrL       0.0       0.0     0.000000   
2 2020-05-15   4136001  9kRcWv60rDACzjR       0.0       0.0  3075.333333   
3 2020-05-15   4136001  Et9kgGMDl729KT4       0.0       0.0   269.933333   
4 2020-05-15   4136001  IQ2d7wF4YD8zU1Q       0.0       0.0  3177.000000   

    TOTAL_YIELD     SOURCE_KEY_y  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  \
0  2.429011e+06  iq8k7ZNt4Mwm3w0            27.004764           25.060789   
1  1.215279e+09  iq8k7ZNt4Mwm3w0            27.004764           25.060789   
2  2.247720e+09  iq8k7ZNt4Mwm3w0            27.004764           25.060789   
3  1.704250e+06  iq8k7ZNt4Mwm3w0            27.004764           25.060789   
4  1.994153e+07  iq8k7ZNt4Mwm3w0            27.004764           25.060789   

   IRRADIATION  
0          0.0  
1          0.0  
2          0.0  
3          0

In [4]:
selected_features = merged_data[['DC_POWER', 'AC_POWER', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']]

corr_matrix = selected_features.corr()

heatmap_data = corr_matrix.values.tolist()

In [5]:
dashboard = lc.Dashboard(
    rows=2,  
    columns=3,  
    theme=lc.Themes.Dark
)

def create_scatter_chart(dashboard, title, x_values, y_values, xlabel, ylabel, column_index, row_index):
    chart = dashboard.ChartXY(
        column_index=column_index,
        row_index=row_index
    )
    chart.set_title(title)

    scatter_series = chart.add_point_series()
    scatter_series.add(x_values, y_values)
    scatter_series.set_point_color(lc.Color(0, 0, 255, 128))

    chart.get_default_x_axis().set_title(xlabel)
    chart.get_default_y_axis().set_title(ylabel)
create_scatter_chart(
    dashboard,
    'AC Power vs. Time',
    merged_data['DATE_TIME'].dt.hour + merged_data['DATE_TIME'].dt.minute / 60,
    merged_data['AC_POWER'],
    'Time (hours)',
    'AC Power (kW)',
    column_index=0,
    row_index=0
)

create_scatter_chart(
    dashboard,
    'AC Power vs. Irradiation',
    merged_data['IRRADIATION'],
    merged_data['AC_POWER'],
    'Irradiation (Wm^-2)',
    'AC Power (kW)',
    column_index=1,
    row_index=0
)

create_scatter_chart(
    dashboard,
    'AC Power vs. Module Temperature',
    merged_data['MODULE_TEMPERATURE'],
    merged_data['AC_POWER'],
    'Module Temperature (Celsius)',
    'AC Power (kW)',
    column_index=2,
    row_index=0
)

create_scatter_chart(
    dashboard,
    'AC Power vs. Ambient Temperature',
    merged_data['AMBIENT_TEMPERATURE'],
    merged_data['AC_POWER'],
    'Ambient Temperature (Celsius)',
    'AC Power (kW)',
    column_index=0,
    row_index=1
)

create_scatter_chart(
    dashboard,
    'AC Power vs. Daily Yield',
    merged_data['DAILY_YIELD'],
    merged_data['AC_POWER'],
    'Daily Yield (Wh)',
    'AC Power (kW)',
    column_index=1,
    row_index=1
)

create_scatter_chart(
    dashboard,
    'AC Power vs. Total Yield',
    merged_data['TOTAL_YIELD'],
    merged_data['AC_POWER'],
    'Total Yield (Wh)',
    'AC Power (kW)',
    column_index=2,
    row_index=1
)

dashboard.open('browser')

127.0.0.1 - - [27/Aug/2024 19:19:49] "GET / HTTP/1.1" 200 -


In [8]:
# features = ['DAILY_YIELD', 'TOTAL_YIELD', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION', 'AC_POWER']
# source_keys = merged_data['SOURCE_KEY_x'].unique()

# chart = lc.ChartXY(
#     theme=lc.Themes.White,
#     title='Correlation Heatmap of Solar Plant Features'
# )

# series = chart.add_heatmap_grid_series(
#     columns=len(heatmap_data),
#     rows=len(heatmap_data[0])
# )

# series.hide_wireframe()
# series.set_intensity_interpolation(False)
# series.invalidate_intensity_values(heatmap_data)

# series.set_palette_colors(
#     steps=[
#         {"value": -1.0, "color": lc.Color(0, 0, 255)},  # Blue for negative correlation
#         {"value": 0.0, "color": lc.Color(255, 255, 255)},  # White for no correlation
#         {"value": 1.0, "color": lc.Color(255, 0, 0)}  # Red for positive correlation
#     ],
#     look_up_property='value',
#     percentage_values=False
# )

# x_axis = chart.get_default_x_axis()
# x_axis.set_title('Feature Index')
# x_axis.set_interval(0, len(selected_features.columns))

# y_axis = chart.get_default_y_axis()
# y_axis.set_title('Feature Index')
# y_axis.set_interval(0, len(selected_features.columns))

# chart.open('browser')


In [5]:
features = merged_data[['IRRADIATION', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE']]
target = merged_data['AC_POWER']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "LightGBM": LGBMRegressor(random_state=42),
    "CatBoost": CatBoostRegressor(verbose=0, random_state=42)
}


In [6]:
dashboard = lc.Dashboard(
    columns=3,
    rows=2,
    theme=lc.Themes.Dark
)

def add_feature_importance_to_dashboard(dashboard, model_name, importances, column_index, row_index):
    """
    Add a feature importance bar chart to the dashboard.
    """
    importance_df = pd.DataFrame({'Feature': features.columns, 'Importance': importances})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    chart = dashboard.BarChart(
        column_index=column_index,
        row_index=row_index,
        row_span=1,
        column_span=1
    )
    chart.set_title(f'{model_name} Feature Importances')
    
    bar_data = [{'category': str(row['Feature']), 'value': float(row['Importance'])} for _, row in importance_df.iterrows()]
    chart.set_data(bar_data)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features.columns)
    ])

for i, (model_name, model) in enumerate(models.items()):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    pipeline.fit(X_train, y_train)

    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)  
    else:
        importances = np.zeros(len(features.columns))

    add_feature_importance_to_dashboard(
        dashboard=dashboard,
        model_name=model_name,
        importances=importances,
        column_index=i % 3,
        row_index=i // 3
    )

estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(random_state=42)),
    ('lgbm', LGBMRegressor(random_state=42)),
    ('cat', CatBoostRegressor(verbose=0, random_state=42))
]

voting_reg = VotingRegressor(estimators=estimators)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', voting_reg)
])

pipeline.fit(X_train, y_train)

ensemble_importances = np.mean([
    pipeline.named_steps['regressor'].estimators_[i].feature_importances_
    for i in range(len(pipeline.named_steps['regressor'].estimators_))
], axis=0)

add_feature_importance_to_dashboard(
    dashboard=dashboard,
    model_name='Ensemble Methods',
    importances=ensemble_importances,
    column_index=2,
    row_index=1
)

dashboard.open('browser')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000255 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 54158, number of used features: 3
[LightGBM] [Info] Start training from score 241.709585
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 54158, number of used features: 3
[LightGBM] [Info] Start training from score 241.709585


127.0.0.1 - - [28/Aug/2024 15:28:57] "GET / HTTP/1.1" 200 -


In [10]:
dashboard = lc.Dashboard(
    columns=3,
    rows=2,
    theme=lc.Themes.Dark
)

def add_prediction_vs_actual_to_dashboard(dashboard, model_name, model, column_index, row_index):
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_test_float = y_test.astype('float64')
    y_pred_float = y_pred.astype('float64')
    
    chart = dashboard.ChartXY(column_index=column_index, row_index=row_index, column_span=1, row_span=1)
    chart.set_title(f'{model_name} Predictions vs Actual')
    
    pred_series = chart.add_point_series()
    pred_series.add(y_test_float.tolist(), y_pred_float.tolist()).set_name('Predicted vs Actual').set_point_size(2)
    
    line_series = chart.add_line_series()
    min_val = min(min(y_test_float), min(y_pred_float))
    max_val = max(max(y_test_float), max(y_pred_float))
    line_series.add([min_val, max_val], [min_val, max_val])
    line_series.set_name('Ideal Line')
    line_series.set_line_color(lc.Color(255, 102, 102))
    
    chart.get_default_x_axis().set_title('Actual AC Power')
    chart.get_default_y_axis().set_title('Predicted AC Power')
    
    legend = chart.add_legend(horizontal=False)
    legend.add(pred_series)
    legend.add(line_series)


preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features.columns)
    ])

for i, (model_name, model) in enumerate(models.items()):
    add_prediction_vs_actual_to_dashboard(dashboard, model_name, model, column_index=i % 3, row_index=i // 3)

estimators = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(random_state=42)),
    ('lgbm', LGBMRegressor(random_state=42)),
    ('cat', CatBoostRegressor(verbose=0, random_state=42))
]

voting_reg = VotingRegressor(estimators=estimators)

add_prediction_vs_actual_to_dashboard(dashboard, 'Ensemble Methods', voting_reg, column_index=2, row_index=1)

dashboard.open('browser')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000139 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 54158, number of used features: 3
[LightGBM] [Info] Start training from score 241.709585
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000201 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 54158, number of used features: 3
[LightGBM] [Info] Start training from score 241.709585


127.0.0.1 - - [28/Aug/2024 15:49:28] "GET / HTTP/1.1" 200 -


In [12]:
features = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
X = merged_data[features]
y = merged_data['AC_POWER']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

dashboard = lc.Dashboard(theme=lc.Themes.Dark, rows=2, columns=2)

ambient_gauge = dashboard.GaugeChart(row_index=0, column_index=0)
ambient_gauge.set_title('Ambient Temperature')
ambient_gauge.set_angle_interval(start=225, end=-45)
ambient_gauge.set_interval(start=0, end=50)
ambient_gauge.set_value_indicators([
    {'start': 0, 'end': 10, 'color': lc.Color(0, 0, 255)},  # Blue
    {'start': 10, 'end': 20, 'color': lc.Color(0, 255, 255)},  # Cyan
    {'start': 20, 'end': 30, 'color': lc.Color(0, 255, 0)},  # Green
    {'start': 30, 'end': 40, 'color': lc.Color(255, 255, 0)},  # Yellow
    {'start': 40, 'end': 50, 'color': lc.Color(255, 0, 0)}  # Red
])
ambient_gauge.set_bar_thickness(100)
ambient_gauge.set_value_indicator_thickness(20)

module_gauge = dashboard.GaugeChart(row_index=0, column_index=1)
module_gauge.set_title('Module Temperature')
module_gauge.set_angle_interval(start=225, end=-45)
module_gauge.set_interval(start=0, end=50)
module_gauge.set_value_indicators([
    {'start': 0, 'end': 10, 'color': lc.Color(0, 0, 255)},  # Blue
    {'start': 10, 'end': 20, 'color': lc.Color(0, 255, 255)},  # Cyan
    {'start': 20, 'end': 30, 'color': lc.Color(0, 255, 0)},  # Green
    {'start': 30, 'end': 40, 'color': lc.Color(255, 255, 0)},  # Yellow
    {'start': 40, 'end': 50, 'color': lc.Color(255, 0, 0)}  # Red
])
module_gauge.set_bar_thickness(100)
module_gauge.set_value_indicator_thickness(20)

power_chart = dashboard.ChartXY(row_index=1, column_index=0, column_span=2, title='Predicted AC Power Over Time')
line_series = power_chart.add_line_series().set_name('Predicted AC Power')

power_chart.get_default_x_axis().set_title('Time (Seconds)')
power_chart.get_default_y_axis().set_title('Predicted AC Power')

def generate_random_weather_data():
    return {
        'AMBIENT_TEMPERATURE': np.random.uniform(X['AMBIENT_TEMPERATURE'].min(), X['AMBIENT_TEMPERATURE'].max()),
        'MODULE_TEMPERATURE': np.random.uniform(X['MODULE_TEMPERATURE'].min(), X['MODULE_TEMPERATURE'].max()),
        'IRRADIATION': np.random.uniform(X['IRRADIATION'].min(), X['IRRADIATION'].max())
    }

def update_dashboard():
    predicted_values = []
    time_values = []
    start_time = time.time()

    for i in range(10000):  
        random_weather = generate_random_weather_data()
        random_weather_df = pd.DataFrame([random_weather])

        predicted_power = model.predict(random_weather_df)[0]

        current_time = time.time() - start_time
        predicted_values.append(predicted_power)
        time_values.append(current_time)

        line_series.add(time_values[-1], predicted_values[-1])

        ambient_gauge.set_value(random_weather['AMBIENT_TEMPERATURE'])
        module_gauge.set_value(random_weather['MODULE_TEMPERATURE'])

        time.sleep(0.5)

dashboard.open(method='browser',live=True)
update_dashboard()




KeyboardInterrupt: 